Celem ćwiczenia jest prezentacja możliwości wykorzystania metod uczenia maszynowego
w rozpoznawaniu komend głosowych.

Materiałem użytym w ćwiczeniu będą próbki nagrań komend z poprzednich ćwiczeń.
Przypomnijmy, w zbiorze jest:
- 30 (+szum) komend,
- nagranych ok. 1,7-2,4 tys. razy.

Treścią zadania jest klasyfikacja komend:
- klasa (class) = komenda,
- próbka (sample) = nagranie.

Ćwiczenie podzielone jest na cztery etapy:
1. Ekstrakcja cech z próbek
2. Podział próbek na treningowe i testowe
3. Uczenie klasyfikatora
4. Klasyfikacja nieznanej próbki
5. Ocena działania klasyfikatora


In [20]:
# import bibliotek
import os
import librosa
import pickle
import sklearn
import numpy

#### 1. Ekstrakcja cech z próbek
Lista ekstraktorów cech: https://librosa.github.io/librosa/feature.html

In [21]:
# ścieżka z folderami z nagraniami - próbki:
train_audio_path = 'input/speech_commands_v0.01'
commands = os.listdir(train_audio_path) 
commands=commands[1:31]


In [22]:
# użyjemy MFCC, tj. Mel-frequency cepstral coefficients
# jednym z parametrów jest liczba wyjściowych współczynników:
n_mfcc = 40

# wczytujemy każde nagranie, wykonujemy ekstrakcję i zapamiętujemy jej wynik
classes = []
class_size = 100 # użyjemy tylko po kilka danych
samples = []
labels = []
print("\nClass: ", end = '')

for command in commands:
    classes.append(command)
    print(command, end = ', ')
    sample_num = 0
    for file in os.listdir(train_audio_path + '/' + command + '/'):
        sample_num += 1
        # przykład z MFCC:
        # y - waveform, sr - sampling rate, n_mfcc - number of MFCCs to return
        y, sr = librosa.load(train_audio_path + '/' + command + '/' + file)
        mfcc_seq = librosa.feature.mfcc(y = y, sr = sr, n_mfcc = n_mfcc)
        mean_mfcc_seq = []
        for feature in mfcc_seq:
            mean_mfcc_seq.append(numpy.mean(feature))
        samples.append(mean_mfcc_seq)
        labels.append(command)
        if sample_num == class_size:
            break

# warto podglądnąć, dla mniejszych n_mfcc, np. 5 co kryje mfcc_seq i mean_mfcc_seq



Class: bed, bird, cat, dog, down, eight, five, four, go, happy, house, left, marvin, nine, no, off, on, one, right, seven, sheila, six, stop, three, tree, two, up, wow, yes, zero, 

In [ ]:
# zapiszmy bazę do późniejszej pracy
database = open('database.pkl', 'wb')
pickle.dump([classes, samples, labels], database)
database.close()
        

#### 2. Wybór danych treningowych i testowych


In [ ]:
# Zautomatyzowany i losowy sposób to użycie:
# "from sklearn.model_selection import train_test_split" oraz funkcji "train_test_split()"

# wczytanie danych
database = open('database.pkl', 'rb')
classes, samples, labels = pickle.load(database)
database.close()

# wybierzmy 10 pierwszych próbek z każdej klasy jako uczące
# oraz kolejne 2 z każdej klasy jako testowe
# uwaga! train_size + test_size NIE może przekroczyć class_size (poprzedni punkt)
train_size = 40
test_size = 5
train_samples = []
train_labels = []
test_samples = []
test_labels = []
for classname in classes:
    train_size_index = 0
    test_size_index = 0
    for i in range(len(labels)):
        if labels[i] == classname and train_size_index < train_size:
            train_samples.append(samples[i])
            train_labels.append(labels[i])
            train_size_index += 1
        elif labels[i] == classname and test_size_index < test_size:
            test_samples.append(samples[i])
            test_labels.append(labels[i])
            test_size_index += 1
        if train_size_index == train_size and test_size_index == test_size:
            break
            
# proszę sprawdzić, czy dane zostały wybrane prawidłowo



#### 3. Uczenie klasyfikatora danymi treningowymi

Lista klasyfikatorów: https://stackabuse.com/overview-of-classification-methods-in-python-with-scikit-learn/

Prosty tutorial: https://www.digitalocean.com/community/tutorials/how-to-build-a-machine-learning-classifier-in-python-with-scikit-learn

In [ ]:

# skalowanie/normalizacja danych
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_samples)
train_samples = scaler.transform(train_samples)
test_samples = scaler.transform(test_samples)

# uczenie
from sklearn.neighbors import KNeighborsClassifier
k = 5
classifier = KNeighborsClassifier(n_neighbors = k)
classifier.fit(train_samples, train_labels)



#### 4. Klasyfikacja nieznanych próbek

In [ ]:


predicted_labels = classifier.predict(test_samples)
    
# zobaczmy wynik - zakomentować dla dużych zbiorów
for i in range (len(test_labels)):
    print(test_labels[i], predicted_labels[i])
#print()



In [ ]:

# 5. Ocena klasyfikatora
# print(test_labels == predicted_labels)
quality = numpy.sum(test_labels == predicted_labels) / len(test_labels) * 100;
print('Ocena klasyfikatora: %.2f %%' % quality)